# KNN with Coulumb matrix

In [1]:
import numpy as np
import pandas as pd
import json
from dscribe.descriptors import CoulombMatrix
from ase import Atoms
import matplotlib.pyplot as plt


test = pd.read_json('data/test.json')
train = pd.read_json('data/train.json')
## Transform atoms entry to ASE atoms object
train.atoms = train.atoms.apply(lambda x: Atoms(**x)) # OBS This one is important!
test.atoms = test.atoms.apply(lambda x: Atoms(**x))

species = []
number_of_atoms = []
atomic_numbers = []
for atom in pd.concat([train.atoms,test.atoms]):
    species = list(set(species+atom.get_chemical_symbols()))
    atomic_numbers = list(set(atomic_numbers+list(atom.get_atomic_numbers())))
    number_of_atoms.append(len(atom))

max_number_of_atoms = np.max(number_of_atoms)
min_atomic_number = np.min(atomic_numbers)
max_atomic_number = np.max(atomic_numbers)


In [2]:
# Setting up the CM descriptor
cm = CoulombMatrix(
    n_atoms_max=max_number_of_atoms,
)

In [3]:
cmats = np.zeros((len(train),max_number_of_atoms**2))
for i,atoms in enumerate(train.atoms):
    if i%1000 == 0:
        print(i)
    cmats[i,:] = cm.create(atoms)
print(len(cmats))

0
1000
2000
3000
4000
5000
6000
7000
8000


In [4]:
cmats.shape

(8000, 400)

In [5]:
X = pd.DataFrame(data = cmats, index=train.id)
y = train['hform']
print('X: {}'.format(X.shape))
print('y: {}'.format(y.shape))

X: (8000, 400)
y: (8000,)


In [9]:
from sklearn.decomposition import PCA
n_comp_PCA = 200

pca = PCA(n_components = n_comp_PCA).fit(X)
X_PCA = pca.transform(X)
print("With {} PCA components {var:0.4f}% of the variance is explained".format(n_comp_PCA, var = 100*np.sum(pca.explained_variance_ratio_)))
print('X_train: {}'.format(X_PCA.shape))


With 200 PCA components 99.9999% of the variance is explained
X_train: (8000, 200)


In [10]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection
from sklearn.metrics import mean_squared_error as mse
def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

CV = model_selection.KFold(10, shuffle=True, random_state=2)


optimal_nn_list = []
min_error_list = []
rmse_v = np.zeros(20)

for train_index, val_index in CV.split(X_PCA,y):
    for l in range(1,21):
        X_train = X_PCA[train_index]
        X_val = X_PCA[val_index]
        y_train = y[train_index]
        y_val = y[val_index]
        model = KNeighborsRegressor(n_neighbors = l, weights='distance')
        model.fit(X_train,y_train)
        test_pred = model.predict(X_val)
        rmse_v[l-1] = rmse(y_val, test_pred)
    
    min_error = np.min(rmse_v)
    opt_nn = np.argmin(rmse_v) + 1
    optimal_nn_list.append(opt_nn)
    min_error_list.append(min_error)

In [11]:
optimal_nn_list

[5, 4, 5, 5, 6, 5, 5, 7, 5, 4]

In [12]:
min_error_list

[0.4733316201196923,
 0.5013205605168346,
 0.47954231645931816,
 0.46984353346122454,
 0.49044445506832574,
 0.4821212625705616,
 0.46149912699128304,
 0.43742870610937107,
 0.4637541511591978,
 0.4631896398980064]

In [13]:
optimal_nn_list[np.argmin(min_error_list)]

7

In [14]:
# Try a model with just one split and compute RMSE, with 7 neigbours a sthat was deemed optimal in the cross validation
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=251)

model = KNeighborsRegressor(n_neighbors = 7, weights='distance')
model.fit(X_train,y_train)
train_prediction = model.predict(X_train)
test_prediction = model.predict(X_test)

print('Train RMSE = {:.2f}'.format(rmse(y_train,train_prediction)))
print('Test RMSE = {:.2f}'.format(rmse(y_test,test_prediction)))


Train RMSE = 0.00
Test RMSE = 0.51
